In [11]:
from src.experiment import load_mind_data_with_neg
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

#######################################################
# 1) Load GRV Data (Item Popularity Over Time)
#######################################################

def load_cox_data_and_survival(cox_data_csv, cox_survival_csv, itemHourLog_csv):
    cox_df = pd.read_csv(cox_data_csv)
    if "T_i0" in cox_df.columns:
        t0_map = dict(zip(cox_df["item_id"], cox_df["T_i0"]))
    else:
        hour_df = pd.read_csv(itemHourLog_csv)
        tmp = hour_df.groupby("item_id")["hour_offset"].min().reset_index()
        t0_map = dict(zip(tmp["item_id"], tmp["hour_offset"]))

    surv_df = pd.read_csv(cox_survival_csv)
    grv_cols = [c for c in surv_df.columns if c.startswith("GRV_t")]
    
    def parse_off(col):
        return int(col.split("t")[-1])

    item_grv = {}
    for row in surv_df.itertuples(index=False):
        it = getattr(row, "item_id")
        d = {parse_off(c): getattr(row, c) for c in grv_cols}
        item_grv[it] = d

    cox_map = {it_id: {"T_i0": t0_map.get(it_id, 0), "grv_map": item_grv[it_id]} for it_id in item_grv}
    return cox_map

def get_grv(cox_map, item_id, current_hour, default_val=0.0):
    if item_id not in cox_map:
        return default_val
    T0 = cox_map[item_id]["T_i0"]
    offset = int(current_hour - T0)
    if offset <= 0:
        return 0.0
    grv_map = cox_map[item_id]["grv_map"]
    offsets = sorted(grv_map.keys())
    offset = min(max(offset, offsets[0]), offsets[-1])
    return grv_map.get(offset, default_val)

#######################################################
# 2) GRU4Rec Model (Session-Based Recommendation)
#######################################################

class GRU4Rec(nn.Module):
    def __init__(self, num_items, emb_dim=16, hidden_size=16, num_layers=1, dropout=0.2):
        super().__init__()
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.gru = nn.GRU(input_size=emb_dim, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, num_items)
        nn.init.xavier_uniform_(self.item_emb.weight)
        nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, session_seq):
        embedded = self.item_emb(session_seq)
        gru_out, _ = self.gru(embedded)
        last_out = gru_out[:, -1, :]
        last_out = self.dropout(last_out)
        return self.fc(last_out)

class SessionDataset(Dataset):
    def __init__(self, df, item2idx, session_length=5):
        self.sessions = []
        grouped = df.groupby("user_id")
        for _, group in grouped:
            group = group.sort_values("time")
            items = group["item_id"].values
            if len(items) < session_length:
                continue
            for i in range(len(items) - session_length + 1):
                session_seq = items[i: i + session_length]
                indices = [item2idx[it] for it in session_seq if it in item2idx]
                if len(indices) == session_length:
                    self.sessions.append(indices)
        self.sessions = np.array(self.sessions)

    def __len__(self):
        return len(self.sessions)

    def __getitem__(self, idx):
        session = self.sessions[idx]
        return torch.tensor(session[:-1], dtype=torch.long), torch.tensor(session[-1], dtype=torch.long)

#######################################################
# 3) Training & Evaluation for GRU4Rec
#######################################################

def train_one_epoch_gru4rec(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for inputs, target in loader:
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        logits = model(inputs)
        loss = loss_fn(logits, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * inputs.size(0)
    return total_loss / len(loader.dataset)

def eval_one_epoch_gru4rec(model, loader, loss_fn, device):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, target in loader:
            inputs, target = inputs.to(device), target.to(device)
            logits = model(inputs)
            loss = loss_fn(logits, target)
            total_loss += loss.item() * inputs.size(0)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == target).sum().item()
    return total_loss / len(loader.dataset), correct / len(loader.dataset)

#######################################################
# 4) GRV-Based Ranking for GRU4Rec
#######################################################

def evaluate_gru4rec_ranking(model, test_df, item2idx, cox_map, gamma=0.3, K=10, device="cpu"):
    """
    Evaluate GRU4Rec performance using HR@K, NDCG@K, Coverage@K, and New Item Coverage@K.
    """
    test_df = test_df.copy()
    test_df["time_hr"] = (test_df["time"] // 3600).astype(int)

    grouped = test_df.groupby("user_id")
    coverage_items = set()
    new_item_hits = 0  # Number of users who received at least one new item

    hits_at_k = 0
    ndcg_at_k = 0  # ✅ Track cumulative NDCG
    total_positives = 0
    all_users = list(grouped.groups.keys())
    all_item_ids = list(item2idx.keys())

    rng = np.random.default_rng(0)

    for user_id in tqdm(all_users, desc="Evaluating GRU4Rec with GRV"):
        g = grouped.get_group(user_id)
        t_hr = g["time_hr"].min()
        pos_items = g[g["label"] == 1]["item_id"].unique()
        candidate_items = np.unique(np.concatenate([pos_items, rng.choice(all_item_ids, size=50, replace=False)]))

        valid_candidates = [(it, item2idx[it]) for it in candidate_items if it in item2idx]
        if not valid_candidates:
            continue

        item_indices = [idx for _, idx in valid_candidates]
        item_ids = [it for it, _ in valid_candidates]

        # Compute model scores
        model.eval()
        with torch.no_grad():
            inputs = torch.tensor(item_indices, dtype=torch.long, device=device).unsqueeze(0)
            logits = model(inputs)
        base_scores = logits.cpu().numpy().flatten()

        # Compute final scores using GRV
        final_scores = []
        for i, (it, base_score) in enumerate(zip(item_ids, base_scores)):
            grv_val = get_grv(cox_map, it, t_hr)
            final_score = (1 - gamma) * base_score + gamma * grv_val
            final_scores.append(final_score)

        # Select top-K items
        top_indices = np.argsort(-np.array(final_scores))[:K]
        top_items = [item_ids[i] for i in top_indices]

        # **Coverage Calculation**
        coverage_items.update(top_items)

        # **New Item Coverage Calculation**
        new_items = set(it for it in top_items if it in cox_map and cox_map[it]["T_i0"] >= 100)
        if new_items:
            new_item_hits += 1  # Count users who received at least one new item

        # **HR Calculation**
        total_positives += len(pos_items)
        hits = sum(1 for pos_it in pos_items if pos_it in top_items)
        hits_at_k += hits

        # **NDCG Calculation**
        dcg = 0.0
        for pos_it in pos_items:
            if pos_it in top_items:
                rank = np.where(np.array(top_items) == pos_it)[0][0] + 1
                dcg += 1 / np.log2(rank + 1)  # ✅ Compute DCG

        # Ideal DCG (iDCG) - best possible ranking
        idcg = sum(1.0 / np.log2(i + 2) for i in range(len(pos_items))) if len(pos_items) > 0 else 0
        ndcg_at_k += dcg / idcg if idcg > 0 else 0  # ✅ Normalize DCG

    # **Normalize Metrics**
    hr = hits_at_k / total_positives if total_positives > 0 else 0
    ndcg = ndcg_at_k / len(all_users) if len(all_users) > 0 else 0  # ✅ Normalize by total users
    coverage = len(coverage_items) / len(item2idx)  # ✅ Normalize by total items
    new_item_coverage = new_item_hits / len(all_users)  # ✅ Normalize by total users

    return hr, ndcg, coverage, new_item_coverage

#######################################################
# 5) Main Experiment for GRU4Rec with GRV
#######################################################

def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    train_df, val_df, test_df = load_mind_data_with_neg("train.csv", "dev.csv", "test.csv")
    item2idx = {i: idx for idx, i in enumerate(pd.concat([train_df["item_id"], val_df["item_id"], test_df["item_id"]]).unique())}

    train_ds = SessionDataset(train_df, item2idx)
    train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)

    model = GRU4Rec(len(item2idx)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    for ep in range(3):
        train_one_epoch_gru4rec(model, train_loader, optimizer, loss_fn, device)

    cox_map = load_cox_data_and_survival("./cox_output/cox_data.csv", "./cox_output/cox_survival.csv", "./output/itemHourLog.csv")

    # ✅ FIX: Unpacking the correct number of values
    hr, ndcg, cov, newcov = evaluate_gru4rec_ranking(model, test_df, item2idx, cox_map, gamma=0.3, K=10, device=device)

    print(f"[RESULT] HR@10={hr:.4f}, NDCG@10={ndcg:.4f}, coverage@10={cov:.4f}, new_item_coverage@10={newcov:.4f}")

if __name__ == "__main__":
    main()

Evaluating GRU4Rec with GRV: 100%|██████████| 57900/57900 [01:17<00:00, 747.01it/s]

[RESULT] HR@10=0.2094, NDCG@10=0.1139, coverage@10=0.9992, new_item_coverage@10=0.9834
